<a href="https://colab.research.google.com/github/Giffy/MongoDB_PyMongo_Tutorial/blob/master/2_Basic_PyMongo_guide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic PyMongo guide


*   Connection to Mongo
*   Basic database, collection management
*   CRUD  one document
*   CRUD  many documents


#1. System setup

##1.1 Install PyMongo (if required)

In [4]:
# Install PyMongo
! python -m pip install pymongo==3.7.2

##1.2 Import PyMongo and set database

In [0]:
import datetime                           # Imports datetime library
import pymongo                            # Imports PyMongo library
from pymongo import MongoClient           # Imports MongoClient 

# uri (uniform resource identifier) defines the connection parameters 
# uri = 'mongodb:// USER : PASSWORD @ SERVER_NAME : PORT / DATABASENAME')
# uri = 'mongodb:// USER : PASSWORD @ SERVER_NAME : PORT / DATABASE_NAME, CLUSTER_1_NAME : PORT , CLUSTER_2_NAME : PORT')
uri = 'localhost:27017'
# start client to connect to MongoDB server 
client = MongoClient( uri )

In [14]:
client.list_database_names()               # From previous module, we will have already a database called 'people'

['local', 'barcelona', 'people']

In [0]:
client.drop_database('people')             # We delete previous module data

##1.3 Download database and import dataset into mongodb

In [16]:
# Downloading JSON with Agenda
!wget https://raw.githubusercontent.com/Giffy/Personal_dataset_repository/master/contacts.json
  
# Uploading data to Mongo Database
!mongodb-linux-x86_64-debian71-3.0.15/bin/mongoimport --jsonArray --db people --collection addressbook  /content/contacts.json

# Reference https://www.kenwalger.com/blog/nosql/mongodb/importing-data-mongoimport/

--2018-11-21 10:41:04--  https://raw.githubusercontent.com/Giffy/Personal_dataset_repository/master/contacts.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 549538 (537K) [text/plain]
Saving to: ‘contacts.json.3’

contacts.json.3     100%[===================>] 536.66K  --.-KB/s    in 0.06s   

2018-11-21 10:41:04 (8.40 MB/s) - ‘contacts.json.3’ saved [549538/549538]

2018-11-21T10:41:05.771+0000	connected to: localhost
2018-11-21T10:41:06.721+0000	imported 1000 documents


#2. Database overview

In [17]:
db = client.people                        # Set the database to work on
db.list_collection_names()                # List the collections available

['addressbook', 'system.indexes']

In [0]:
collection = db.addressbook               # Set the collection to work on

##2.1 Dataset size and attributes

In [19]:
## Dataset content summary
num_documents = collection.count_documents({'_id' : {'$exists' : 1}})
attributes = list (collection.find().limit(1)[1])     ## WARNING use as reference , NoSQL db can have different attributes by document

print ('Number of documents : %d' % num_documents)
print ('Attributes names : %s' % attributes)

Number of documents : 1000
Attributes names : ['_id', 'index', 'name', 'isActive', 'registered', 'age', 'gender', 'eyeColor', 'favoriteFruit', 'company', 'tags']


In [20]:
## Dataset content summary
def content_attribute( attribute_name ):
  return collection.distinct( attribute_name )

for attribute_name in attributes:
    content =  content_attribute( attribute_name )
    sample = content
    if len(content) > 20:
      sample = content[:20]
    print ('Item name : ' + attribute_name + 
           '\n   Unique content : ' + str(len(content)) +
           '\n   Content : ' + str(sample) )

Item name : _id
   Unique content : 1000
   Content : [ObjectId('5bf536410f50e03768521ee8'), ObjectId('5bf536410f50e03768521ee9'), ObjectId('5bf536410f50e03768521eea'), ObjectId('5bf536410f50e03768521eeb'), ObjectId('5bf536410f50e03768521eec'), ObjectId('5bf536410f50e03768521eed'), ObjectId('5bf536410f50e03768521eee'), ObjectId('5bf536410f50e03768521eef'), ObjectId('5bf536410f50e03768521ef0'), ObjectId('5bf536410f50e03768521ef1'), ObjectId('5bf536410f50e03768521ef2'), ObjectId('5bf536410f50e03768521ef3'), ObjectId('5bf536410f50e03768521ef4'), ObjectId('5bf536410f50e03768521ef5'), ObjectId('5bf536410f50e03768521ef6'), ObjectId('5bf536410f50e03768521ef7'), ObjectId('5bf536410f50e03768521ef8'), ObjectId('5bf536410f50e03768521ef9'), ObjectId('5bf536410f50e03768521efa'), ObjectId('5bf536410f50e03768521efb')]
Item name : index
   Unique content : 1000
   Content : [1, 2, 0, 3, 4, 6, 5, 7, 8, 9, 11, 12, 10, 13, 14, 16, 17, 15, 18, 19]
Item name : name
   Unique content : 1000
   Content : ['K

In [21]:
## We can list one document
list ( collection.find().limit(1) )

[{'_id': ObjectId('5bf536410f50e03768521ee8'),
  'age': 38,
  'company': {'email': 'kittysnow@digitalus.com',
   'location': {'address': '154 Arlington Avenue', 'country': 'Italy'},
   'phone': '+1 (949) 568-3470',
   'title': 'DIGITALUS'},
  'eyeColor': 'blue',
  'favoriteFruit': 'apple',
  'gender': 'female',
  'index': 1,
  'isActive': False,
  'name': 'Kitty Snow',
  'registered': datetime.datetime(2018, 1, 23, 4, 46, 15),
  'tags': ['ut', 'voluptate', 'consequat', 'consequat']}]

#3. Create Read Update Delete - Part 2

##3.1 Adding and delete attributes

### Add new atribute to your collection

In [0]:
collection.update_many( {"age" :{ "$gte" :0 }}, {"$set" : { "favoriteColor" : "red" }})

In [0]:
collection.update_many( {"age" :{ "$nin" : [""] }}, {"$set" : { "favoriteBook" : "Harry Potter" }})

In [0]:
list(collection.find({"age" : 20} , {"favoriteColor","favoriteBook", "name","age"}).limit(1))

[{'_id': ObjectId('5bf1b58d85cb96520a4ac106'),
  'age': 20,
  'favoriteBook': 'Harry Potter',
  'name': 'Aurelia Gonzales'}]

### Delete attribute and data

In [0]:
## Deprecated   collection.update( {"age" :{ "$gte" :0 }}, {"$unset" :{ "favoriteColor" :1 }}, {multi :1})
collection.update_many( {"age" :{ "$gte" :0 }}, {"$unset" :{ "favoriteColor" :1 , "favoriteBook" :1}})

In [0]:
list(collection.find({"age" : 20} , {"favoriteColor","favoriteBook", "name","age"}).limit(1))

[{'_id': ObjectId('5bf1b58d85cb96520a4ac106'),
  'age': 20,
  'name': 'Aurelia Gonzales'}]

##3.2 Data visualization and Queries

###3.2.1 Visualization of attribute names  -  first level

In [22]:
print (list (collection.find({"age" : 38, "gender" : 'female'}).limit(1)[1]))

['_id', 'index', 'name', 'isActive', 'registered', 'age', 'gender', 'eyeColor', 'favoriteFruit', 'company', 'tags']


###3.2.2 Visualization of attribute unique content (sorted)

In [30]:
query = collection.distinct( "age" )
print ("Age :" + str( sorted( query ) ))

query = collection.distinct( "gender" ) 
print ("Gender :" + str( sorted( query ) ))

Age :[20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]
Gender :['female', 'male']


- Exercise:  Check the unique eye colors and favorite fruit

      hint: attributes =  eyeColor and favoriteFruit

###3.2.3 Find document by id

In [57]:
# Get the id of existing document

documents = collection.find( {"_id": {"$exists": True}} , ['name','age']).limit(1)

itemId = ""
for item in documents:
  itemId = str( item['_id'] )

print (itemId) 

5bf536410f50e03768521ee8


In [0]:
# First import objectid object
from bson.objectid import ObjectId

In [59]:
list(collection.find({"_id": ObjectId( itemId )} , ['name','age','favoriteFruit','company.email']))   ##  reading second level attribute 'email'

[{'_id': ObjectId('5bf536410f50e03768521ee8'),
  'age': 38,
  'company': {'email': 'kittysnow@digitalus.com'},
  'favoriteFruit': 'apple',
  'name': 'Kitty Snow'}]

###3.2.4 Filter by fields

In [63]:
# collection.find(  FILTER_CONDITION   ,  FIELDS_to_retreive )
# FIELDS      1 field:      string
#             more than 1 : a list [] 

filters = {"isActive": True}
fields = ['name','age', 'isActive','company.email']

list(collection.find( filters , fields ).limit(1))

[{'_id': ObjectId('5bf536410f50e03768521eeb'),
  'age': 39,
  'company': {'email': 'karynrhodes@rodemco.com'},
  'isActive': True,
  'name': 'Karyn Rhodes'}]

In [64]:
print( collection.count_documents( filters ) )

516


In [73]:
# Multiple filters

filters = {"$or": [{"age" : 28}, {"age" : 29}] , "gender" : 'female'}

print( collection.count_documents ( filters ) ) # count in Mongo the found documents 
print( len (list (collection.find( filters ))) ) # count in Python the found documents

36
36


###3.2.5 Find by regex ( name starts with ' Ki* ')

In [76]:
import re
regex = re.compile('^Ki', re.IGNORECASE)

# find documents which ssn starts with Ki

filters = { 'name' : regex }
fields = { '_id' : 0, 'name' : 1, 'isActive' : 1, 'age' : 1 }     #  Hide _id in reply  

list ( collection.find( filters , fields ) )

[{'age': 38, 'isActive': False, 'name': 'Kitty Snow'},
 {'age': 20, 'isActive': True, 'name': 'Kimberly House'},
 {'age': 34, 'isActive': False, 'name': 'Kirk Walsh'},
 {'age': 21, 'isActive': True, 'name': 'Kirby Buckley'},
 {'age': 22, 'isActive': True, 'name': 'Kinney Wynn'},
 {'age': 38, 'isActive': True, 'name': 'Kirkland Buckner'},
 {'age': 40, 'isActive': True, 'name': 'Kidd Arnold'},
 {'age': 22, 'isActive': False, 'name': 'Kimberley Chase'}]

###3.2.6 Sort query Ascending

In [77]:
# Ascending
list ( collection.find( filters , fields ).sort('age', pymongo.ASCENDING) )

[{'age': 20, 'isActive': True, 'name': 'Kimberly House'},
 {'age': 21, 'isActive': True, 'name': 'Kirby Buckley'},
 {'age': 22, 'isActive': True, 'name': 'Kinney Wynn'},
 {'age': 22, 'isActive': False, 'name': 'Kimberley Chase'},
 {'age': 34, 'isActive': False, 'name': 'Kirk Walsh'},
 {'age': 38, 'isActive': False, 'name': 'Kitty Snow'},
 {'age': 38, 'isActive': True, 'name': 'Kirkland Buckner'},
 {'age': 40, 'isActive': True, 'name': 'Kidd Arnold'}]

In [80]:
# Descending 
list ( collection.find( filters , fields ).sort('age', pymongo.DESCENDING) )

[{'age': 40, 'isActive': True, 'name': 'Kidd Arnold'},
 {'age': 38, 'isActive': False, 'name': 'Kitty Snow'},
 {'age': 38, 'isActive': True, 'name': 'Kirkland Buckner'},
 {'age': 34, 'isActive': False, 'name': 'Kirk Walsh'},
 {'age': 22, 'isActive': True, 'name': 'Kinney Wynn'},
 {'age': 22, 'isActive': False, 'name': 'Kimberley Chase'},
 {'age': 21, 'isActive': True, 'name': 'Kirby Buckley'},
 {'age': 20, 'isActive': True, 'name': 'Kimberly House'}]

#4. Comparison Query Operators & Count

###4.1 Count documents with "age"    equal   to 38

In [0]:
collection.count_documents({"age": 38})
# the previous method was : collection.find({"age": 38}).count()  (deprecated)

###4.2 Count documents with numbers :  "greater than or equal", "greater than",  "equal", "little than", "little than or equal"

In [88]:
collection.count_documents({"age": {"$gte" : 38}})

152

In [89]:
collection.count_documents({"age": {"$gt" : 38}})

103

In [90]:
collection.count_documents({"age": {"$eq" : 38}})

49

In [0]:
collection.count_documents({"age": {"$lt" : 38}})

In [0]:
collection.count_documents({"age": {"$lte" : 38}})

In [0]:
collection.count_documents({"age": {"$gte" : 0 }})

In [0]:
collection.count_documents({"age" : { "$nin" : [""] } })

In [0]:
collection.find().count()

###4.3 Count documents with letters :  "greater than or equal"  or  "little than or equal"

In [93]:
# Da el mismo resultado gte que gt 
collection.count_documents({"name": {"$gt" : "B" }})

944

In [94]:
# Da el mismo resultado lte que lt 
collection.count_documents({"name": {"$lt" : "B" }})

56

In [0]:
# Como es con D ?

collection.count_documents({"name": '/B/' })

### Visualize MAXIMUM and MINIMUM content

In [0]:
max( collection.distinct( "age" )) 

In [0]:
min( collection.distinct( "age" )) 

In [0]:
agemale = collection.find({"gender" : 'male'}).distinct( "age" )
agefemale = collection.find({"gender" : 'female'}).distinct( "age")

print ('Male -  Min age: ' + str(min(agemale)) + ' and Max age: ' + str(max(agemale)))
print ('Female -  Min age: ' + str(min(agefemale)) + ' and Max age: ' + str(max(agefemale)))

### Operators  IN and  NIN

In [0]:
## Deprecated 
# print( collection.find( {"name" : { "$in": ['Mercado Riddle', "Kitty Snow" ] }} ).count())
# print( collection.find( {"name" : { "$nin": ['Mercado Riddle', "Kitty Snow" ] }} ).count())

In [0]:
print( collection.count_documents( { "name" : { "$in": ['Mercado Riddle', "Kitty Snow" ] }} ))    # includes the names in count
print( collection.count_documents( { "name" : { "$nin": ['Mercado Riddle', "Kitty Snow" ] }} ))   # excludes the names in count

In [0]:
list( collection.find( { "name" : { "$in": ['Mercado Riddle', "Kitty Snow" ] }}, ''))      # includes the names in find

# Logical Query Operators

### AND

In [0]:
list ( collection.find({
                        "$and":[
                            {"name" : "Kitty Snow"},
                            {"age": 38}]
                       }, '') # ,'' --> Muestra del ID
     )

### OR

In [0]:
print ( collection.find({
                          "$or":[
                              {"age" : 28}, 
                              {"age" : 29}]
                        }, '').count()     # ,'' --> Muestra del ID
      )

###AND & OR

In [0]:
list ( collection.find({
            "$and":[
                    {"$or":[
                         {"name" : "Kitty Snow"}, 
                         {"name" : 'Mercado Riddle'}
                    ]},
                    {"age": 38}
              ]}, '' )   # ,'' --> Muestra del ID 
     )

# Exists & not exists

In [0]:
# Counts documents with attribute 'age'

# print(collection.find({'age' : {'$exists' : 1}}).count())   ## Count is deprecated. Use Collection.count_documents instead.
print(collection.count_documents({'_id' : {'$exists' : 1}}))

In [0]:
import datetime
collection.insert_one( {
  'company': {'email': 'jordigonzalez@yurture.com',
   'location': {'address': '694 Hewes Street', 'country': 'USA'},
   'phone': '+1 (940) 501-3963',
   'title': 'YURTURE'},
  'eyeColor': 'green',
  'favoriteFruit': 'banana',
  'gender': 'male',
  'index': 0,
  'isActive': False,
  'name': 'Jordi Gonzalez',
  'registered': datetime.datetime(2015, 2, 11, 4, 22, 39),
  'tags': ['enim', 'id', 'velit', 'ad', 'consequat']
})

In [0]:
# Counts documents without attribute 'age'
print(collection.count_documents({'age' : {'$exists' : 0}}))

## Delete one or multiple items by ID

In [0]:
to_delete = list(collection.find({'company.email' : 'jordigonzalez@yurture.com'}, '_id'))
len(to_delete)

In [0]:
i=0
for doc in to_delete:
  id = doc.get('_id')
  collection.delete_one({'_id': id})
  i +=1
  print("Deleted " + str(i))

# List items that belong to a list or not

In [0]:
# Count documents with age equal to 28, 29 or 30
print(collection.count_documents({'age' : {'$in': [ 28, 29, 30]}}))

In [0]:
# Count documents with age different to 28, 29 or 30
print(collection.count_documents({'age' : {'$nin': [ 28, 29, 30]}}))

In [0]:
# Count documents with favorite fruit different to banana and apple
print(collection.count_documents({'favoriteFruit' : {'$nin': [ 'banana', 'apple']}}))


# Indexes

### Create index in Background

In [0]:
# MONGO    collection.create_index({"name":1}, {"unique": True} )
collection.create_index([( "age" , pymongo.ASCENDING)])

In [0]:
# MONGO    db.people.createIndex({age:1},{name: "customAgeIndex"})

# Modify type of attribute in all documents

In [0]:
from bson.code import Code

mapper = Code("""
               function () {
                 this.age.forEach(function(z) {
                   emit(z);
                 });
               }
               """)

modification = Code("""
                    function (key) {
                      newKey = "1";                                 
                     return newKey;
                     }
                 """)


collection.map_reduce(
         mapper,
         modification,
         "results", full_response=True)


In [0]:
'''// De Float o Integer a String
collection.find({"age": {$exists: true}}).forEach(function(doc){
  if(doc.Lon != null){
      if (Number.isInteger(doc.Lon)) {
        doc.Lon = doc.Lon.toString();            
      } else {
        doc.Lon = doc.Lon.toString();            
      }
      db.people.people.save(doc);
  } // End of If Condition
}) // End of foreach


// De String a Float
collection.find({"age": {$exists: true}}).forEach(function(doc){
  if(doc.Lon != null){
     if(doc.Lon.length > 0){
        var newCost = doc.Lon.replace(/,/g, '').replace(/\$/g, '');
        var costString = parseFloat(newCost).toFixed(10);
        doc.Lon = parseFloat(costString);    
        db.baltimore.save(doc);
     } // End of 2 If Condition
  } // End of 1 If Condition
}) // End of foreach
'''

In [0]:

'''
// De Float o Integer a String
db.baltimore.find({"street" : {$nin:["Caton Ave1"]}, "Lon": {$exists: true}}).forEach(function(doc){
  if(doc.Lon != null){
      if (Number.isInteger(doc.Lon)) {
        doc.Lon = doc.Lon + ".0"
      } else {
        doc.Lon = doc.Lon.toString();            
      }
      db.baltimore.save(doc);
  } // End of If Condition
}) // End of foreach


// De String a Float
db.baltimore.find({"street" : {$nin:["Caton Ave1"]}, "Lon": {$exists: true}}).forEach(function(doc){
  if(doc.Lon != null){
     if(doc.Lon.length > 0){
        var newCost = doc.Lon.replace(/,/g, '').replace(/\$/g, '');
        var costString = parseFloat(newCost).toFixed(10);
        doc.Lon = parseFloat(costString);    
        db.baltimore.save(doc);
     } // End of 2 If Condition
  } // End of 1 If Condition
}) // End of foreach


'''

#  Under construction

In [0]:
#collection.update_many( {"age" :{ "$nin" : [""] }}, {"$set" : {"$project" : {"AnoNacimiento" : {"$subtract": [2018, '$age']} }}})

db = collection.aggregate ( [ {"$project" :{"AnoNaciminto":{"$subtract": [2018, '$age']} }}])
list (db)[:2]


In [0]:
#collection.update_many( {"age" :{ "$nin" : [""] }}, {"$set" : {collection.aggregate ( [ {"$project" :{ "AnoNaciminto" : {"$subtract": [2018, '$age']} }}])}})


In [0]:
collection.update_many( {"age" :{ "$nin" : [""] }}, {"$set" : { "AnoNacimiento" : 20144 } })

In [0]:
collection.aggregate([ {$group: {""}}])

In [0]:
collection.update_many( {"AnoNacimiento"}, {"$rename" : { "AnoNacimient"} })

# Group by 

In [0]:
list ( collection.aggregate([ 
            {'$group':{
                  '_id' :{'IsActive':"$isActive", 'Age':"$age"}
                      }
            } ,{'$limit':5} ]))    ##  '$limit':5   shows only 5 items

In [0]:
list ( collection.aggregate([ 
            {'$group':{
                  '_id' :{'IsActive':"$isActive", 'Age':"$age"}     ###  Creates ID with IsActive + Age
                      }
            } ]))[:5]        ##  shows only 5 items but all 1000 documents are downloaded   #### WARNING not recommendable

In [0]:
list ( collection.aggregate([
              {'$group':{
                    '_id' :{'IsActive':"$isActive", 'Age':"$age"}      ###  Creates ID with IsActive + Age
                    ,'age2' : {'$max' : '$age'}                        ###  Adds a new attribute with Age content  ($max is used to pick up the content)
                    ,'isActive2' : {'$min' : '$isActive'}              ###  Adds a new attribute with isActive content 
                        } 
              }
              ,{'$limit': 5}
              ])
     )

In [0]:
list ( collection.aggregate([
              {'$group':{
                    '_id' :{'IsActive':"$isActive", 'Age':"$age"}
                    ,'age2' : {'$max' : '$age'}
                    ,'isActive2' : {'$min' : '$isActive'} 
                        } 
              }
              ,{'$project' : {'_id' : 0 , 'age2': 1 , 'isActive2' :1 }}
              ,{'$limit': 5}
              ])
     )

# Group by &  Sort

In [0]:
list ( collection.aggregate([
              {'$group':{
                    '_id' :{'IsActive':"$isActive", 'Age':"$age"}
                    ,'age2' : {'$max' : '$age'}
                    ,'isActive2' : {'$min' : '$isActive'} 
                        } 
              }
              ,{'$project' : {'_id' : 0 , 'age2': 1 , 'isActive2' :1 }}
              ,{'$sort': {'age2':1 , 'isActive2' : 1}}
              ,{'$limit': 5}
              ])
     )

In [0]:


#collection.find({"_id": {"$exists": True}} , ['name','age','company.email']).skip(1).limit(4).sort({"name": -1 }))

#collection.find().sort({'age': '-1' })

#list ( collection.aggregate([{'$group':{'_id' :{'Name':"$name"}}},{'$sort':{'name':1}}]))
# list ( collection.aggregate([{'$group':{'_id' :{'Age':"$age"}}},{'$sort':{'age':1}}]))
#list ( collection.aggregate([{'$group':{'_id' :{'IsActive':"$isActive", 'Age':"$age"}}},{'$sort':{'isActive':-1}}]))
#list(	collection.aggregate([{'$group':{'_id' :{'pais':"$country", 'npais':"$country_txt"}}},{'$sort':{'country':-1}},{'$limit':5}])))
#list ( collection.aggregate([{'$group':{'_id' : {'Age':"$age"}}},{'$sort':{'isActive':-1}}]))

#collection.find ({'age' : 28}, {'name'}).sort ([{ 'name', pymongo.ASCENDING}]) 
#list(collection.find ({'age' : 28}, {'name'}))



#list ( collection.aggregate([{'$group':{'_id' :[{'IsActive':"$isActive", 'Age':"$age"},{'$sum' :1}] }},{'$sort':{'age':-1}},{'$limit':5} ]))



#collection.find ({'age' : 28}, {'name'}).sort ([{ 'name', pymongo.ASCENDING}]) 


#collection.find ({'age' : 28}, {'name'}).sort ([{ 'name', pymongo.ASCENDING}]) 




#collection.find().sort({'$age': 1})
#list ( collection.find().sort({'age': '-1' }) )

list ( collection.aggregate([
              {'$group':{
                    '_id' :{'IsActive':"$isActive", 'Age':"$age"}
              , 'age2' : {'$max' : '$age'}, 'isActive2' : {'$max' : '$isActive'} 
                  } 
#              , {'_id' : False }
              }
              , {'$project' : {'_id' : 0 , 'age2': 1 , 'isActive2' :1 }}
              
              ,{'$sort': {'age2':1 , 'isActive2' : 1}}
#             ,{'$limit': 10}
#             ,{'$sort': {'IsActive':-1}}             
              ])
     )






# Bulk Writes

In [0]:
collection.bulkWrite([
    { update_one( {'company': {'email': 'jordigonzalez@yurture.com', 'location': {'address': '694 Hewes Street', 'country': 'USA'}, 'phone': '+1 (940) 501-3963', 'title': 'YURTURE'},
                   'name': 'Jordi Gonzalez',
                   'gender': 'male'
    })},
    { update_one( {'company': {'email': 'pepegonzalez@yurture.com', 'location': {'address': '694 Hewes Street', 'country': 'USA'}, 'phone': '+1 (940) 501-3963', 'title': 'YURTURE'},
                   'name': 'Pepe Gonzalez',
                   'gender': 'male'
    })},
    { update_one( {'company': {'email': 'davidgonzalez@yurture.com', 'location': {'address': '694 Hewes Street', 'country': 'USA'}, 'phone': '+1 (940) 501-3963', 'title': 'YURTURE'},
                   'name': 'David Gonzalez',
                   'gender': 'male'
    })} 
    
    
])

NameError: ignored

In [0]:
requests = [
  InsertOne({ '_id': 11, 'name': 'Edgar Martinez', 'salary': "8.5M" }),    # Insert #1
  InsertOne({ '_id': 3, 'name': 'Alex Rodriguez', 'salary': "18.3M" }),    # Insert #2
  InsertOne({ '_id': 24, 'name': 'Ken Griffey Jr.', 'salary': "12.4M" }),  # Insert #3
  InsertOne({ '_id': 11, 'name': 'David Bell', 'salary': "2.5M" }),        # Insert #4
  InsertOne({ '_id': 19, 'name': 'Jay Buhner', 'salary': "5.1M" })         # Insert #5
]

response = employees.bulk_write(requests)

NameError: ignored